In [104]:
import os
os.chdir('../')
%pwd

'D:\\Python\\Industry level\\NLP'

In [105]:
from dataclasses import dataclass
from pathlib import Path
from sentiment_analysis.utils.common import read_yaml,create_directories

In [106]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [107]:
from sentiment_analysis.constants import *
from pathlib import Path

In [108]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILEPATH
    ):
        self.config = read_yaml(config_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= Path(config.root_dir),
            source_URL= config.source_URL,
            local_data_file= Path(config.local_data_file),
            unzip_dir= Path(config.unzip_dir)
        )

        return data_ingestion_config


In [109]:
import requests
from zipfile import ZipFile
import os
import sys
from sentiment_analysis.logger.logger_setup import logger
from sentiment_analysis.exception.customException import CustomException

In [110]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        try:
            data_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs(os.path.dirname(zip_download_dir), exist_ok=True)

            logger.info(f"Downloading data from {data_url} into file {zip_download_dir}")
        
            response = requests.get(data_url, stream=True)
            response.raise_for_status()
        
            with open(zip_download_dir, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
                
            logger.info(f"Download completed and saved to {zip_download_dir}")
        except Exception as e:
            raise CustomException(e, sys)
        
    def extract_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [111]:
os.chdir(r'D:\Python\Industry level\NLP\NLP-IMDB-sentiment-analysis-End-to-end')
%pwd


'D:\\Python\\Industry level\\NLP\\NLP-IMDB-sentiment-analysis-End-to-end'

In [112]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_file()
except Exception as e:
    raise CustomException(e,sys)